In [ ]:
#!pip install simpletransformers

In [ ]:
#conda install pytorch torchvision -c pytorch

In [ ]:
#!pip install -U sentence-transformers

In [13]:
import json,glob,nltk,copy,time,pickle #sentence_transformers,torch
import numpy as np
from scipy import spatial

from queue import PriorityQueue
#from sentence_transformers import SentenceTransformer,util
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bhavika\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
import glob
data = []

files = glob.glob('comm_use_subset/*', recursive=True)
number_of_articles = len(files)
bound = 1000

for single_file in files[0:bound]:
    with open(single_file, 'r') as f:
        json_file = json.load(f)
    
    # Retrieve title
        title = json_file["metadata"]["title"]

    # Retrieve abstracts
        abstracts = []
        if len(json_file["abstract"]) != 0 :
            for abstract in json_file["abstract"]:
                abstracts.append(abstract["text"])

    # Retrieve texts
        texts = []
        for text in json_file["body_text"]:
            texts.append(text["text"])

        data.append([title,abstracts,texts])

In [15]:
import pandas as pd
df = pd.DataFrame(data,columns = ["title","abstract","body"])

In [16]:
df

,title,abstract,body
0,Supplementary Information An eco-epidemiologic...,[],[- Figure S1 : Phylogeny of all sequences belo...
1,immunity to pathogens taught by specialized hu...,[Dendritic cells (DCs) are specialized antigen...,[iNTRODUCTiON Human beings are constantly expo...
2,Public Health Responses to and Challenges for ...,[Dengue has a negative impact in low-and lower...,[Pathogens and vectors can now be transported ...
3,a section of the journal Frontiers in Pediatri...,"[Fifteen years ago, United Nations world leade...",[In addition to preventative care and nutritio...
4,MINI REVIEW Function of the Deubiquitinating E...,[Posttranslational modification of proteins by...,[Ubiquitination is a widely used posttranslati...
...,...,...,...
995,Breastfeeding and Respiratory Infections in th...,[Background: Viral respiratory tract infection...,[Respiratory tract infections are a leading ca...
996,Evolutionary and Transmission Dynamics of Reas...,[H5N1 highly pathogenic avian influenza (HPAI)...,[The H5N1 highly pathogenic avian influenza (H...
997,"Mortality, Severe Acute Respiratory Infection,...",[Introduction: While there is much information...,"[Early during the 2009 pandemic, the number of..."
998,Probability of introducing porcine epidemic di...,[Background: Porcine epidemic diarrhea virus (...,[Porcine epidemic diarrhea (PED) is a pig dise...


In [17]:
import numpy as np
for i in range(len(df)):
    if df['abstract'][i] == []:
        df['abstract'][i] = np.nan
    if df['body'][i] == []:
        df['body'][i] = np.nan
    

In [18]:
for i in range(len(df.abstract)):
    if df.abstract[i] == []:
        df['abstract'][i] = " ".join(map(str,df['abstract'][i]))
        


In [19]:
for i in range(len(df.abstract)):
    if df.body[i] == []:
        df['body'][i] = " ".join(map(str,df['body'][i]))

In [20]:
df.isnull().sum()

title        0
abstract    92
body         0
dtype: int64

In [21]:
df = df.dropna()

In [24]:
df = df.reset_index()

In [11]:
df = df.drop(['index'] , axis=1)

In [25]:
df

,index,title,abstract,body
0,1,immunity to pathogens taught by specialized hu...,[Dendritic cells (DCs) are specialized antigen...,[iNTRODUCTiON Human beings are constantly expo...
1,2,Public Health Responses to and Challenges for ...,[Dengue has a negative impact in low-and lower...,[Pathogens and vectors can now be transported ...
2,3,a section of the journal Frontiers in Pediatri...,"[Fifteen years ago, United Nations world leade...",[In addition to preventative care and nutritio...
3,4,MINI REVIEW Function of the Deubiquitinating E...,[Posttranslational modification of proteins by...,[Ubiquitination is a widely used posttranslati...
4,5,Evaluation of the tuberculosis programme in Ni...,[Background: Tuberculosis is a devastating dis...,[The Ministry of Health of the People's Republ...
...,...,...,...,...
903,994,Enhanced pathogenicity of low-pathogenic H9N2 ...,"[How to cite this article: Ismail ZM, El-Deeb ...",[Avian influenza (AI) is a respiratory disease...
904,995,Breastfeeding and Respiratory Infections in th...,[Background: Viral respiratory tract infection...,[Respiratory tract infections are a leading ca...
905,996,Evolutionary and Transmission Dynamics of Reas...,[H5N1 highly pathogenic avian influenza (HPAI)...,[The H5N1 highly pathogenic avian influenza (H...
906,997,"Mortality, Severe Acute Respiratory Infection,...",[Introduction: While there is much information...,"[Early during the 2009 pandemic, the number of..."


In [26]:
for i in range(len(df.abstract)):
        df['abstract'][i] = " ".join(map(str,df['abstract'][i]))
        

C:\Users\Bhavika\AppData\Local\Temp/ipykernel_10384/705503033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['abstract'][i] = " ".join(map(str,df['abstract'][i]))


In [27]:
for i in range(len(df.abstract)):
        df['body'][i] = " ".join(map(str,df['body'][i]))

C:\Users\Bhavika\AppData\Local\Temp/ipykernel_10384/472310026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['body'][i] = " ".join(map(str,df['body'][i]))


In [28]:
df

,index,title,abstract,body
0,1,immunity to pathogens taught by specialized hu...,Dendritic cells (DCs) are specialized antigen-...,iNTRODUCTiON Human beings are constantly expos...
1,2,Public Health Responses to and Challenges for ...,Dengue has a negative impact in low-and lower ...,Pathogens and vectors can now be transported r...
2,3,a section of the journal Frontiers in Pediatri...,"Fifteen years ago, United Nations world leader...",In addition to preventative care and nutrition...
3,4,MINI REVIEW Function of the Deubiquitinating E...,Posttranslational modification of proteins by ...,Ubiquitination is a widely used posttranslatio...
4,5,Evaluation of the tuberculosis programme in Ni...,Background: Tuberculosis is a devastating dise...,The Ministry of Health of the People's Republi...
...,...,...,...,...
903,994,Enhanced pathogenicity of low-pathogenic H9N2 ...,"How to cite this article: Ismail ZM, El-Deeb A...",Avian influenza (AI) is a respiratory disease ...
904,995,Breastfeeding and Respiratory Infections in th...,Background: Viral respiratory tract infections...,Respiratory tract infections are a leading cau...
905,996,Evolutionary and Transmission Dynamics of Reas...,H5N1 highly pathogenic avian influenza (HPAI) ...,The H5N1 highly pathogenic avian influenza (HP...
906,997,"Mortality, Severe Acute Respiratory Infection,...",Introduction: While there is much information ...,"Early during the 2009 pandemic, the number of ..."


In [29]:
df['cleaned_title']=df['title'].apply(lambda x:x.lower())
df['cleaned_abstract']=df['abstract'].apply(lambda x:x.lower())
df['cleaned_body']=df['body'].apply(lambda x:x.lower())



In [30]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

In [31]:
import re
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

In [32]:
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Expanding Contractions
df['cleaned_title']=df['cleaned_title'].apply(lambda x:expand_contractions(x))
df['cleaned_abstract']=df['cleaned_abstract'].apply(lambda x:expand_contractions(x))
df['cleaned_body']=df['cleaned_body'].apply(lambda x:expand_contractions(x))

In [33]:
def clean_text(text):
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    #text=re.sub('[^a-z]',' ',text)
    return text

df['cleaned_title']=df['cleaned_title'].apply(lambda x:clean_text(x))
df['cleaned_abstract']=df['cleaned_abstract'].apply(lambda x:clean_text(x))
df['cleaned_body']=df['cleaned_body'].apply(lambda x:clean_text(x))

In [34]:
df['cleaned_title']=df['cleaned_title'].apply(lambda x: re.sub(' +',' ',x))
df['cleaned_abstract']=df['cleaned_abstract'].apply(lambda x: re.sub(' +',' ',x))
df['cleaned_body']=df['cleaned_body'].apply(lambda x: re.sub(' +',' ',x))

In [ ]:
!pip install spacy

In [ ]:
spacy install en

In [36]:
df

,index,title,abstract,body,cleaned_title,cleaned_abstract,cleaned_body
0,1,immunity to pathogens taught by specialized hu...,Dendritic cells (DCs) are specialized antigen-...,iNTRODUCTiON Human beings are constantly expos...,immunity to pathogens taught by specialized hu...,dendritic cells (dcs) are specialized antigen-...,introduction human beings are constantly expos...
1,2,Public Health Responses to and Challenges for ...,Dengue has a negative impact in low-and lower ...,Pathogens and vectors can now be transported r...,public health responses to and challenges for ...,dengue has a negative impact in low-and lower ...,pathogens and vectors can now be transported r...
2,3,a section of the journal Frontiers in Pediatri...,"Fifteen years ago, United Nations world leader...",In addition to preventative care and nutrition...,a section of the journal frontiers in pediatri...,"fifteen years ago, united nations world leader...",in addition to preventative care and nutrition...
3,4,MINI REVIEW Function of the Deubiquitinating E...,Posttranslational modification of proteins by ...,Ubiquitination is a widely used posttranslatio...,mini review function of the deubiquitinating e...,posttranslational modification of proteins by ...,ubiquitination is a widely used posttranslatio...
4,5,Evaluation of the tuberculosis programme in Ni...,Background: Tuberculosis is a devastating dise...,The Ministry of Health of the People's Republi...,evaluation of the tuberculosis programme in ni...,background: tuberculosis is a devastating dise...,the ministry of health of the people is republ...
...,...,...,...,...,...,...,...
903,994,Enhanced pathogenicity of low-pathogenic H9N2 ...,"How to cite this article: Ismail ZM, El-Deeb A...",Avian influenza (AI) is a respiratory disease ...,enhanced pathogenicity of low-pathogenic h9n2 ...,"how to cite this article: ismail zm, el-deeb a...",avian influenza (ai) is a respiratory disease ...
904,995,Breastfeeding and Respiratory Infections in th...,Background: Viral respiratory tract infections...,Respiratory tract infections are a leading cau...,breastfeeding and respiratory infections in th...,background: viral respiratory tract infections...,respiratory tract infections are a leading cau...
905,996,Evolutionary and Transmission Dynamics of Reas...,H5N1 highly pathogenic avian influenza (HPAI) ...,The H5N1 highly pathogenic avian influenza (HP...,evolutionary and transmission dynamics of reas...,h5n1 highly pathogenic avian influenza (hpai) ...,the h5n1 highly pathogenic avian influenza (hp...
906,997,"Mortality, Severe Acute Respiratory Infection,...",Introduction: While there is much information ...,"Early during the 2009 pandemic, the number of ...","mortality, severe acute respiratory infection,...",introduction: while there is much information ...,"early during the 2009 pandemic, the number of ..."


In [37]:
df.to_csv("cleaned_filee_nolemma_1000.csv")

In [41]:
import pandas as pd
df = pd.read_csv("cleaned_filee_nolemma_1000.csv")

In [38]:
df = df.dropna()

In [39]:
df = df.reset_index()

In [ ]:
df=df.drop("index", axis=1)

In [42]:
df

,Unnamed: 0,index,title,abstract,body,cleaned_title,cleaned_abstract,cleaned_body
0,0,1,immunity to pathogens taught by specialized hu...,Dendritic cells (DCs) are specialized antigen-...,iNTRODUCTiON Human beings are constantly expos...,immunity to pathogens taught by specialized hu...,dendritic cells (dcs) are specialized antigen-...,introduction human beings are constantly expos...
1,1,2,Public Health Responses to and Challenges for ...,Dengue has a negative impact in low-and lower ...,Pathogens and vectors can now be transported r...,public health responses to and challenges for ...,dengue has a negative impact in low-and lower ...,pathogens and vectors can now be transported r...
2,2,3,a section of the journal Frontiers in Pediatri...,"Fifteen years ago, United Nations world leader...",In addition to preventative care and nutrition...,a section of the journal frontiers in pediatri...,"fifteen years ago, united nations world leader...",in addition to preventative care and nutrition...
3,3,4,MINI REVIEW Function of the Deubiquitinating E...,Posttranslational modification of proteins by ...,Ubiquitination is a widely used posttranslatio...,mini review function of the deubiquitinating e...,posttranslational modification of proteins by ...,ubiquitination is a widely used posttranslatio...
4,4,5,Evaluation of the tuberculosis programme in Ni...,Background: Tuberculosis is a devastating dise...,The Ministry of Health of the People's Republi...,evaluation of the tuberculosis programme in ni...,background: tuberculosis is a devastating dise...,the ministry of health of the people is republ...
...,...,...,...,...,...,...,...,...
903,903,994,Enhanced pathogenicity of low-pathogenic H9N2 ...,"How to cite this article: Ismail ZM, El-Deeb A...",Avian influenza (AI) is a respiratory disease ...,enhanced pathogenicity of low-pathogenic h9n2 ...,"how to cite this article: ismail zm, el-deeb a...",avian influenza (ai) is a respiratory disease ...
904,904,995,Breastfeeding and Respiratory Infections in th...,Background: Viral respiratory tract infections...,Respiratory tract infections are a leading cau...,breastfeeding and respiratory infections in th...,background: viral respiratory tract infections...,respiratory tract infections are a leading cau...
905,905,996,Evolutionary and Transmission Dynamics of Reas...,H5N1 highly pathogenic avian influenza (HPAI) ...,The H5N1 highly pathogenic avian influenza (HP...,evolutionary and transmission dynamics of reas...,h5n1 highly pathogenic avian influenza (hpai) ...,the h5n1 highly pathogenic avian influenza (hp...
906,906,997,"Mortality, Severe Acute Respiratory Infection,...",Introduction: While there is much information ...,"Early during the 2009 pandemic, the number of ...","mortality, severe acute respiratory infection,...",introduction: while there is much information ...,"early during the 2009 pandemic, the number of ..."


In [43]:
df['total_text'] = ''
for i in range(len(df)):
    df['total_text'][i] =  df['cleaned_abstract'][i] + df['cleaned_body'][i]

C:\Users\Bhavika\AppData\Local\Temp/ipykernel_10384/2848797716.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_text'][i] =  df['cleaned_abstract'][i] + df['cleaned_body'][i]


In [44]:
#for total text

from nltk.tokenize import sent_tokenize, word_tokenize
data1 = []
for i in range(500):
    temp = []
    for j in word_tokenize(df['total_text'][i]):
            temp.append(j)
    data1.append(temp)

In [45]:
query = ['What are the coronoviruses?','What was discovered in Wuhuan in December 2019?',
           'What is Coronovirus Disease 2019?','What is COVID-19?','What is caused by SARS-COV2?',
           'How is COVID-19 spread?','Where was COVID-19 discovered?','How does coronavirus spread?']
df1 = pd.DataFrame(query,columns = ['query'])
df1

,query
0,What are the coronoviruses?
1,What was discovered in Wuhuan in December 2019?
2,What is Coronovirus Disease 2019?
3,What is COVID-19?
4,What is caused by SARS-COV2?
5,How is COVID-19 spread?
6,Where was COVID-19 discovered?
7,How does coronavirus spread?


In [46]:
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)



df1['cleaned_query']=df1['query'].apply(lambda x:x.lower())
df1['cleaned_query']=df1['query'].apply(lambda x:x.lower())

# Expanding contractions
df1['cleaned_query']=df1['cleaned_query'].apply(lambda x:expand_contractions(x))
df1['cleaned_query']=df1['cleaned_query'].apply(lambda x:expand_contractions(x))


# Removing extra spaces
df1['cleaned_query']=df1['cleaned_query'].apply(lambda x: re.sub(' +',' ',x))
df1['cleaned_query']=df1['cleaned_query'].apply(lambda x: re.sub(' +',' ',x))

In [47]:
df1

,query,cleaned_query
0,What are the coronoviruses?,what are the coronoviruses?
1,What was discovered in Wuhuan in December 2019?,what was discovered in wuhuan in december 2019?
2,What is Coronovirus Disease 2019?,what is coronovirus disease 2019?
3,What is COVID-19?,what is covid-19?
4,What is caused by SARS-COV2?,what is caused by sars-cov2?
5,How is COVID-19 spread?,how is covid-19 spread?
6,Where was COVID-19 discovered?,where was covid-19 discovered?
7,How does coronavirus spread?,how does coronavirus spread?


In [ ]:
!pip install gensim

In [48]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(data1, vector_size=30, min_count=2,window=5, sg=1,workers=4)

In [49]:
print('Vocabulary size:', len(w2v_model.wv))

Vocabulary size: 40736


In [50]:

import numpy as np
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(30)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.key_to_index:
                embeddings.append(w2v_model.wv.word_vec(tok))
            else:
                embeddings.append(np.random.rand(30))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries

df['vector']=df['total_text'].apply(lambda x: get_embedding_w2v(x.split()))
#df['body_vector']=df['lemmatized_body'].apply(lambda x :get_embedding_w2v(x.split()))
df1['query_vector']=df1['cleaned_query'].apply(lambda x :get_embedding_w2v(x.split()))                                                 

C:\Users\Bhavika\AppData\Local\Temp/ipykernel_10384/3880857190.py:9: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  embeddings.append(w2v_model.wv.word_vec(tok))


In [51]:
df1

,query,cleaned_query,query_vector
0,What are the coronoviruses?,what are the coronoviruses?,"[-0.08999045465733863, -0.3907756048280266, 0...."
1,What was discovered in Wuhuan in December 2019?,what was discovered in wuhuan in december 2019?,"[0.27501655756578136, -0.18426690563412296, 0...."
2,What is Coronovirus Disease 2019?,what is coronovirus disease 2019?,"[0.45807848546112384, 0.09376810017452433, 0.8..."
3,What is COVID-19?,what is covid-19?,"[0.05880513491894612, -0.4048961761343892, 0.4..."
4,What is caused by SARS-COV2?,what is caused by sars-cov2?,"[0.3088704892665147, -0.15185896681145045, 0.5..."
5,How is COVID-19 spread?,how is covid-19 spread?,"[0.19743993382699845, -0.19511126981196708, 0...."
6,Where was COVID-19 discovered?,where was covid-19 discovered?,"[0.26629462693104555, -0.1991539962058708, 0.8..."
7,How does coronavirus spread?,how does coronavirus spread?,"[0.15616440524215414, -0.08257917056173011, 0...."


In [52]:
df

,Unnamed: 0,index,title,abstract,body,cleaned_title,cleaned_abstract,cleaned_body,total_text,vector
0,0,1,immunity to pathogens taught by specialized hu...,Dendritic cells (DCs) are specialized antigen-...,iNTRODUCTiON Human beings are constantly expos...,immunity to pathogens taught by specialized hu...,dendritic cells (dcs) are specialized antigen-...,introduction human beings are constantly expos...,dendritic cells (dcs) are specialized antigen-...,"[-0.03274730659739616, -0.06026944809245253, 0..."
1,1,2,Public Health Responses to and Challenges for ...,Dengue has a negative impact in low-and lower ...,Pathogens and vectors can now be transported r...,public health responses to and challenges for ...,dengue has a negative impact in low-and lower ...,pathogens and vectors can now be transported r...,dengue has a negative impact in low-and lower ...,"[0.2289631519276287, -0.15405362141150233, 0.5..."
2,2,3,a section of the journal Frontiers in Pediatri...,"Fifteen years ago, United Nations world leader...",In addition to preventative care and nutrition...,a section of the journal frontiers in pediatri...,"fifteen years ago, united nations world leader...",in addition to preventative care and nutrition...,"fifteen years ago, united nations world leader...","[0.22731241732948185, -0.2798937210012487, 0.6..."
3,3,4,MINI REVIEW Function of the Deubiquitinating E...,Posttranslational modification of proteins by ...,Ubiquitination is a widely used posttranslatio...,mini review function of the deubiquitinating e...,posttranslational modification of proteins by ...,ubiquitination is a widely used posttranslatio...,posttranslational modification of proteins by ...,"[0.004668452342764712, -0.14642357896136365, 0..."
4,4,5,Evaluation of the tuberculosis programme in Ni...,Background: Tuberculosis is a devastating dise...,The Ministry of Health of the People's Republi...,evaluation of the tuberculosis programme in ni...,background: tuberculosis is a devastating dise...,the ministry of health of the people is republ...,background: tuberculosis is a devastating dise...,"[0.21826305131518137, -0.21210656934713418, 0...."
...,...,...,...,...,...,...,...,...,...,...
903,903,994,Enhanced pathogenicity of low-pathogenic H9N2 ...,"How to cite this article: Ismail ZM, El-Deeb A...",Avian influenza (AI) is a respiratory disease ...,enhanced pathogenicity of low-pathogenic h9n2 ...,"how to cite this article: ismail zm, el-deeb a...",avian influenza (ai) is a respiratory disease ...,"how to cite this article: ismail zm, el-deeb a...","[0.18489937199135145, -0.05464636252392896, 0...."
904,904,995,Breastfeeding and Respiratory Infections in th...,Background: Viral respiratory tract infections...,Respiratory tract infections are a leading cau...,breastfeeding and respiratory infections in th...,background: viral respiratory tract infections...,respiratory tract infections are a leading cau...,background: viral respiratory tract infections...,"[0.21229355280869983, -0.1574627751263944, 0.6..."
905,905,996,Evolutionary and Transmission Dynamics of Reas...,H5N1 highly pathogenic avian influenza (HPAI) ...,The H5N1 highly pathogenic avian influenza (HP...,evolutionary and transmission dynamics of reas...,h5n1 highly pathogenic avian influenza (hpai) ...,the h5n1 highly pathogenic avian influenza (hp...,h5n1 highly pathogenic avian influenza (hpai) ...,"[0.1341498141754168, -0.14361408081461927, 0.5..."
906,906,997,"Mortality, Severe Acute Respiratory Infection,...",Introduction: While there is much information ...,"Early during the 2009 pandemic, the number of ...","mortality, severe acute respiratory infection,...",introduction: while there is much information ...,"early during the 2009 pandemic, the number of ...",introduction: while there is much information ...,"[0.24301760104401968, -0.13019841325100556, 0...."


In [53]:
df1['similarity'] = [list() for i in range(len(df1))]

In [54]:
l1 = []
l1 = df1['query_vector']

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
for qindex,query in enumerate(l1):
    df1['similarity'][qindex] = df['vector'].apply(lambda x: cosine_similarity(np.array(query).reshape(1, -1),np.array(x).reshape(1, -1)).item())
    df1['similarity'][qindex] = df1.similarity[qindex].sort_values(ascending = False)
    df1['similarity'][qindex] = df1.similarity[qindex].head(5)
    print(df1['similarity'][qindex])
    


223    0.906347
855    0.902471
751    0.899518
383    0.897854
92     0.894247
Name: vector, dtype: float64
853    0.926102
594    0.923536
524    0.920462
554    0.917550
679    0.916648
Name: vector, dtype: float64
633    0.859244
666    0.851131
718    0.808704
676    0.808667
601    0.808010
Name: vector, dtype: float64
223    0.802467
751    0.797019
601    0.793065
859    0.788814
383    0.787757
Name: vector, dtype: float64
691    0.844240
207    0.844091
270    0.842794
652    0.837939
880    0.836953
Name: vector, dtype: float64
691    0.929115
859    0.924813
601    0.924516
758    0.922968
244    0.922944
Name: vector, dtype: float64
550    0.883807
907    0.883801
601    0.881823
306    0.880654
619    0.877589
Name: vector, dtype: float64
632    0.793853
745    0.793766
390    0.790921
576    0.790704
691    0.789051
Name: vector, dtype: float64


In [58]:
for i in range(len(df1)):
    print("query: ",df1['query'][i])
    l1 = df1['similarity'][i].index
    print("Relevant docs abstract:   ")
    for i in l1:
        print(df['title'][i],'.')
    print("-------------------------------------------------------------------------------------------------------------------")

query:  What are the coronoviruses?
Relevant docs abstract:   
Challenges with using names to link digital biodiversity information .
Environmental Health Outdoor environments and human pathogens in air .
Dynamic graphs, community detection, and Riemannian geometry .
Antifragility and Tinkering in Biology (and in Business) Flexibility Provides an Efficient Epigenetic Way to Manage Risk .
nan .
-------------------------------------------------------------------------------------------------------------------
query:  What was discovered in Wuhuan in December 2019?
Relevant docs abstract:   
High Prevalence and Putative Lineage Maintenance of Avian Coronaviruses in Scandinavian Waterfowl .
Occurrence and sequence analysis of porcine deltacoronaviruses in southern China .
Clinical Study Viral Aetiology in Adults with Acute Upper Respiratory Tract Infection in Jinan, Northern China .
Complete Genome Sequence of a Novel WU Polyomavirus Isolate from Arkansas, USA, Associated with Acute Respir